In [50]:
import numpy as np
import pandas as pd

In [51]:
attempts_df = pd.read_csv("TextBanking_Efforts.xlsx - Attempts.csv")
contact_df = pd.read_csv("TextBanking_Efforts.xlsx - Contact.csv")

# clean -- values
for column in attempts_df.columns:
    attempts_df = attempts_df[(attempts_df[column] != "--")]
for column in contact_df.columns:
    contact_df = contact_df[(contact_df[column] != "--")]

In [52]:
attempts_df.columns

Index(['timestamp', 'message_direction', 'Voter ID'], dtype='object')

In [53]:
contact_df.columns

Index(['Question Sent', 'question_Response_type', 'response to question',
       'updated_at', 'Voter ID'],
      dtype='object')

In [59]:
# dictionary mapping {Question : {available response : count}}
question_response = {}
total_count = 0

# initialize questions
for question in contact_df["Question Sent"].unique():
    question_response[question] = {}

# initialize available responses per question
for row_idx in range(0, contact_df.shape[0]):
    # 'Question Sent' is col 0, 'response to question' is col 2
    question = contact_df.iloc[row_idx, 0]
    response = contact_df.iloc[row_idx, 2]
    if response not in question_response[question]:
        question_response[question][response] = 1
        total_count += 1
    else:
        question_response[question][response] += 1
        total_count += 1

print(contact_df["Voter ID"].unique().shape[0])
question_response

419


{'Planning to vote?': {'Yes': 200, 'No': 44},
 'Do you have the proper ID to vote?': {'Yes': 49, 'Unsure': 3, 'No': 2},
 'Texting correct person': {'Contacted': 142,
  'Refused': 53,
  'Wrong Number': 111,
  'Other Language': 1,
  'Spanish': 2},
 'Texting the Right Person': {'Wrong Number': 8}}

In [60]:
# Success at getting a response from reaching out, ie contact success rate
all_contacts_df = pd.DataFrame({"Voter ID": attempts_df["Voter ID"].unique()})
all_contacts_df["Contact Success"] = 0 # changed to 1 later if success
all_contacts_df = all_contacts_df.set_index('Voter ID')
all_contacts_df = all_contacts_df.sort_values(by=["Voter ID"])
contact_df = contact_df.sort_values(by=["Voter ID"])

In [69]:
# Define successful contact as someone who was contacted correctly and communicated with correctly

# Voter ID is column 4 in contact_df
prev_voterID = contact_df.iloc[0, 4]
responses = {}
success = False # list of questions this person was asked over text

for row_idx in range(0, contact_df.shape[0]):
    curr_voterID = contact_df.iloc[row_idx, 4]
    # if new person, analyze the boolean and reset boolean
    if curr_voterID != prev_voterID:
        # analyze previous voterID. If success, change in all_contacts_df
        # various booleans
        not_plan_to_vote = 'Planning to vote?' in responses and responses['Planning to vote?']=='No'
        no_proper_ID = 'Do you have the proper ID to vote?' in responses and responses['Do you have the proper ID to vote?']!="Yes"
        not_correct_person = 'Texting correct person' in responses and responses['Texting correct person']!='Contacted'
        not_right_person = 'Texting the Right Person' in responses and responses['Texting the Right Person'] == 'Wrong Number'
        if not_correct_person or not_right_person:
            success = False
        if success:
            all_contacts_df.loc[prev_voterID, "Contact Success"] = 1
        success = True # default boolean to False
        responses = {} # reset responses

    prev_voterID = curr_voterID

all_contacts_df

Contact Success
Voter ID                   
1000205011                0
1000433225                0
1000746850                0
1001614539                0
1004309873                0
...                     ...
2193193194                0
2193193564                0
2193194892                0
2193225791                0
2193229462                0

[10167 rows x 1 columns]

In [70]:
# sanity check to make sure all voter ids in contacts_df are also in all_contacts_df
count = 0
for voter_id in contact_df["Voter ID"].unique():
    if voter_id not in all_contacts_df.index:
        count += 1
count

0

In [71]:
# sanity check for previous block:
num_success = 0
for row_idx in range(0, all_contacts_df.shape[0]):
    if all_contacts_df.iloc[row_idx, 0] == 1:
        num_success += 1
num_success
# should be <200

418

In [72]:
# get final percentage of successfully contacted people
num_success / all_contacts_df.shape[0]

0.0411134061178322